In [1]:
%matplotlib inline
import xarray as xr
import matplotlib.pyplot as plt
import glob
import numpy as np
import cartopy.crs as ccrs
import cartopy
from scipy.stats import linregress
import pandas as pd
from scipy import signal

In [2]:
ds=xr.open_dataset('sst.mnmean.nc')
ds1=ds.sel(time=slice('2008-01-01','2016-01-01'))
ds2=ds.sel(time=slice('2016-03-01','2017-11-01'))
ds3=xr.merge([ds1,ds2])
sst=ds3['sst']


In [5]:
climatology_sst=sst.groupby('time.month').mean('time')
anomalies_sst=sst.groupby('time.month')-climatology_sst

In [7]:
clt_opz=ds['cltcalipso_opaque_z']
opz=ds['zopaque']
diff=(clt_opz)-opz
visible_depth=diff*2

In [6]:
files=sorted(glob.glob('/home/z5241410/Desktop/UNSW/Calipso_data/full_data/Map_OPAQ330m*.nc'))
ds=xr.open_mfdataset(files,combine='nested',concat_dim='time')

In [8]:
climatology_vd=visible_depth.groupby('time.month').mean('time')
anomalies_vd=visible_depth.groupby('time.month')-climatology_vd

In [9]:
x=anomalies_vd
y=anomalies_sst

In [11]:
y_interp = y.interp({'lat': x.latitude, 'lon': x.longitude})

In [12]:
def multi_apply_along_axis(func1d, axis, arrs, *args, **kwargs):
    carrs = np.concatenate(arrs, axis)
    offsets=[]
    start=0
    for i in range(len(arrs)-1):
        start += arrs[i].shape[axis]
        offsets.append(start)
    def helperfunc(x, *args, **kwargs):
        arrs = np.split(x, offsets)
    if helperfunc is None:
        return [np.nan,np.nan]
        return func1d(*[*arrs, *args], **kwargs)
    return np.apply_along_axis(helperfunc, axis, carrs, *args, **kwargs)

In [13]:
from scipy.stats.stats import pearsonr

corr = multi_apply_along_axis(pearsonr, 0,[x,y_interp])
corr.shape

/home/z5241410/miniconda3/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


(90, 180)